# Formation RNAseq CEA - juin 2024

*Enseignantes: Sandrine Caburet et Claire Vandiedonck*

Session IFB : 7 CPU + 33 GB de RAM

R version 4.2.3

# Part 8: Exploratory analysis I (before normalisation)

   

- 0. 1 - Setting up this R session on IFB core cluster
- 0. 2 - Parameters to be set or modified by the user
- 1 - Loading input data and metadata
- 2 - Quality assessment
- 3 - Saving our results

---
## **Before going further**

<div class="alert alert-block alert-danger"><b>Caution:</b> 
Before starting the analysis, save a backup copy of this notebok : in the left-hand panel, right-click on this file and select "Duplicate"<br>
You can also make backups during the analysis. Don't forget to save your notebook regularly: <kbd>Ctrl</kbd> + <kbd>S</kbd> or click on the 💾 icon.
</div>

<div class="alert alert-block alert-warning"><b>Warning:</b>You are strongly advised to run the cells in the indicated order. If you want to rerun cells above, you can just restart the kernel to start at 1 again. </div>

---
---
## 0. Set up parameters and RSession

---

### 0.1 - Setting up this R session on IFB core cluster

<em>loaded JupyterLab</em> : Version 3.5.0

#### **0.1-a. Jupyter session**

Just as in a bash notebook, we can have information about the number of threads available for this session. To do so, we will use the `system()`function in R which executes bash commands.

In [1]:
## Code cell 1 ##

session_parameters <- function(){
    
    jupytersession <- c(system('echo "=== Cell launched on $(date) ==="', intern = TRUE),
                        system('squeue -hu $USER', intern = TRUE))
    
    jobid <- system("squeue -hu $USER | awk '/sys/dash {print $1}'", intern = TRUE)
    jupytersession <- c(jupytersession,
                        "=== Current IFB session size: Medium (5CPU, 21 GB) ===",
                        system(paste("sacct --format=JobID,AllocCPUS,ReqMem,NodeList,Elapsed,State -j", jobid), intern = TRUE))
    print(jupytersession[1:6])
    
    return(invisible(NULL))
}

session_parameters()

[1] "=== Cell launched on Tue Jun 25 15:23:53 CEST 2024 ==="                         
[2] "          40370879      fast sys/dash scaburet  R    5:39:31      1 cpu-node-11"
[3] "=== Current IFB session size: Medium (5CPU, 21 GB) ==="                         
[4] "JobID         AllocCPUS     ReqMem        NodeList    Elapsed      State "      
[5] "------------ ---------- ---------- --------------- ---------- ---------- "      
[6] "40370879              7        33G     cpu-node-11   05:39:31    RUNNING "      


__

#### **0.1-b. R session**

Next we load into this R session the various packages that we will use.


<div class="alert alert-block alert-info"> <b> Info on packages: </b><br>
    
Since this is the first time during this course we are using R packages, the next code cells 2-5 will explain you some basic commands to deal with R packages. </div>

In R we can call packages with some functions to do special analyses.
To call such a package, the usual command is `library(name_of_the_package)`. It works if the package is already installed on your computer/server, otherwise you have to install it from the main R repository called CRAN (https://cran.r-project.org/). The main function to install R packages is `install.packages()`. Some packages are not available via the CRAN package repository, specifically the ones used in bioinformatic analyses, but via another repository: ***Bioconductor***. In order to install a package from Bioconductor, you need to install the `BiocManager` package first, then use its fonction `install()`. An example of the commands to use is provided in Code cell 5.

- We can first identify the directories, called ***libraries***, where these packages are installed. 

In [2]:
## Code cell 2 ##

.libPaths()

[1] "/shared/ifbstor1/home/scaburet/R/x86_64-conda-linux-gnu-library/4.2"
[2] "/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library"

If this is the first time you are using this R version on the IFB core cluster or you never installed packages before, the previous command should return only one folder, readable by all IFB users. It should be : `/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library` since the R version currently installed is 4.2.3.

If you have already installed R packages, you should see another folder in your home directory, most likely with this absolute path: `/shared/ifbstor1/home/mylogin/R/x86_64-conda-linux-gnu-library/4.2` *(for R versions 4.2)*.

- You can see how many **packages were already installed on the IFB core cluster** with the following command:

In [3]:
## Code cell 3 ##

length(list.files("/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library"))  # modifier l'indice selon le repertoire souhaité

[1] 1154

That is an impressive number! It is highly likely that the packages you need were already installed on the server!

- To know **which packages were already installed**, you can either go to the corresponding folder, or use the R function `installed.packages()` which will return all packages installed with a lot of information. The most interesting infos are in the first three columns, with the name, the version and the library path. We can see how it looks like for the first packages with the command `head()`, or verify the presence of a given package with the command `grep()`.

In [4]:
## Code cell 4 ##

head(installed.packages()[,c(1,2,3)])

grep("ggplot2", installed.packages()[,c(1,2,3)], value = TRUE)

,Package,LibPath,Version
ReactomePA,ReactomePA,/shared/ifbstor1/home/scaburet/R/x86_64-conda-linux-gnu-library/4.2,1.42.0
ggfortify,ggfortify,/shared/ifbstor1/home/scaburet/R/x86_64-conda-linux-gnu-library/4.2,0.4.17
reactome.db,reactome.db,/shared/ifbstor1/home/scaburet/R/x86_64-conda-linux-gnu-library/4.2,1.82.0
ADGofTest,ADGofTest,/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library,0.3
ANCOMBC,ANCOMBC,/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library,2.0.2
AUCell,AUCell,/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library,1.20.2


[1] "ggplot2"

- **If your package of interest was not yet installed on the IFB cluster, you have two options:**

1. either you ask them to install it for you *(and therefore for all other users)* using the community forum (https://community.france-bioinformatique.fr/). They are quite reactive and this would be a good option to make sure all dependecies are correctly installed together with your package.

2. or you install it in your home directory.


If you go for the second option because you are in a rush with your analyses, you may skip the above commands and just try to install it, only if needed.

If the `.libPaths()`command run in Code cell 2 above did not return the name of your R library folder in your home, you first have to create it, then load its path as one of the possibilities: 


In [5]:
## Code cell 5 ##

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix (thus only created if not yet present)
dir.create("~/R/x86_64-conda-linux-gnu-library/4.2", recursive = TRUE, showWarnings = FALSE)

# this new directory is added to the possible paths:
.libPaths('~/R/x86_64-conda-linux-gnu-library/4.2')

# and we verify its addition:
.libPaths()

[1] "/shared/ifbstor1/home/scaburet/R/x86_64-conda-linux-gnu-library/4.2"
[2] "/shared/ifbstor1/software/miniconda/envs/r-4.2.3/lib/R/library"

Now we can install new packages in our home, if they are not available on the cluster.    

The following command lines using `require` and a condition with `if()` will install the package only if it is not already there!<br>
Then the packages are loaded into your R session.       
   
<span style="color:red">It is good practice to have such a command at the top of your notebook/script when you are doing analyses in R.</span>

In [ ]:
## Code cell 6 ##

# list the required libraries from the CRAN repository
requiredLib <- c(
    "tidyverse",
    "data.table",
    "ggfortify",
    "ggrepel",
    "RColorBrewer",
    "matrixStats",
    "BiocManager"    
)

# list the required libraries from the Bioconductor project
requiredBiocLib <- c("affy")

# install required libraries if not yet installed
for (lib in requiredLib) {
  if (!require(lib, character.only = TRUE, quiet = TRUE)) {
    install.packages(lib, quiet = TRUE)
  }
}

for( lib in requiredBiocLib) {
  if (!require(lib, character.only = TRUE, quiet = TRUE)) {
  BiocManager::install(lib, quiet = TRUE)
  }
}

# load libraries
message("Loading required libraries")
for (lib in requiredLib) {
  library(lib, character.only = TRUE)}
for (lib in requiredBiocLib) {
  library(lib, character.only = TRUE)}

# remove variables from the R session if they are no longer necessary 
rm(lib, requiredLib, requiredBiocLib)



<div class="alert alert-block alert-warning"><b>Warning:</b><ul>
    <li><b><i>DO NOT worry</i></b> if you see a large red output!! </b> You should see this large red output only once, when the relevant packages are installed in your home directory. (Afterwards, they will be detected as present, and this large red output won't show if you run the cell another time. <il>
 <li><b><i> DO NOT panic </i></b> if you see a warning in the output: such commands return sometimes warnings that can arise due to changes in R versions, but as a general rule the functions in these packages will still work.<il>
     </div>

The next command will return the versions for R and R-packages used in your current R session. You can check that the packages called in a previous command are indeed loaded:

In [ ]:
## Code cell 7 ##   

cat("---Here are the info of my R session with the loaded packages:\n")
sessionInfo()

It looks ok and we can start the analysis!

--

### 0.2 - Parameters to be set or modified by the user
---



- To know our **current working directory**, we use the function ̀`getwd()` (for *get working directory*, equivalent to `pwd` in bash).

In [ ]:
## Code cell 8 ##   

getwd()

<div class="alert alert-block alert-info"> <b> Info on working directory: </b><br>In a Jupyter Hub and a jupyter notebook in R, by default the working directory is where the notebook is opened</div>

- Using a full path with a `/` at the end, **define the folder** of the project as  `gohome` variable, the folder where you work as the `myfolder` variable.

In [ ]:
## Code cell 9 ##

gohome <- "/shared/projects/2413_rnaseq_cea/"
gohome

myfolder <- getwd()
myfolder

- We also define the folder with the raw data as `datafolder`, the reference folder with the reference genome data `reffolder` and we specify the variable of the reference genome annotation used `annot_version`.

In [ ]:
## Code cell 10 ##

datafolder <- paste0(myfolder, "/Data/")
datafolder

reffolder <- paste0(gohome, "alldata/Reference/")
reffolder
annot_version <- "vM35"

- With a `/` at the end, we also add the folder `salmonfolder` with salmon data  we still need to aggregate for all samples at both the transcript and gene levels.

In [ ]:
## Code cell 11 ##

salmonfolder <- paste0(myfolder, "/Results/salmon/")
salmonfolder

- With a `/` at the end, define the path to the folder where the results of this initial exploratory analysis will be stored:

In [ ]:
## Code cell 12 ##

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste0(myfolder,"/Results/pca1/"), recursive = TRUE)

# storing the path to this output folder in a variable
pca1folder <- paste(myfolder,"/Results/pca1/", sep = "")
pca1folder

# listing the content of the folder
print(system(paste("ls -hlt", pca1folder), intern = TRUE) )

(When creating a directory with the `recursive = TRUE`argument, you obtain a red warning if the directory already exists.    
You can inactivate this warning with the additional argument  `showWarnings = FALSE`, as in the Code cell 5 above. ) 

- Last, we specify the size of the graphical outputs that will be used for all the plots in the notebook.    
This setting could be modified at will for each plot. 

In [ ]:
## Code cell 13 ##

options(repr.plot.width = 15, repr.plot.height = 8)

---
## 1 - Loading metadata and input (STAR and Salmon) data
---

<div class="alert alert-block alert-info"><b>Info:</b><br>
In the previous parts of the pipeline, we worked only on three samples. Now that we obtained read counts per features, we have <i>lighter</i> files and we can work on <b> the whole dataset with all 11 samples</b>.</div>

We now need three types of files:   
1. a **metadata** file, providing information about the samples, in particular the conditions of the experiment. 
2. the **gene read counts** produced by `STAR` and `featureCounts` (as in the Pipe_06 notebook), on all 11 samples   
3. the **transcript counts** produced by `SALMON` for each sample

You should have obtained 2 TRUE output, and the files should appear in your folder in the left column.

### 1.1 - Loading metadata   
---

We copy the first metadata file in our personal folder from the alldata folder with the `file.copy()` function.

In [ ]:
## Code cell 14 ##

file.copy(from = paste0(gohome, "alldata/Data/sampleData-GSE158661.tsv"),
         to = datafolder)

We read the tabulated table and we store the data in a dataframe named `samples` in our R session.  
The file has column names, so we set `header=TRUE`, and the separator as a tab (<code>sep = "\t"</code>).

In [ ]:
## Code cell 15 ##

samples <- read.table(paste0(datafolder, "sampleData-GSE158661.tsv"),
                      sep = "\t",
                      header = TRUE,
                      stringsAsFactors = FALSE)

We then look at the class, structure and first rows of the imported file.

In [ ]:
## Code cell 16 ##

class(samples)

In [ ]:
## Code cell 17 ##

str(samples)

In [ ]:
## Code cell 18 ##

head(samples, n = 12)

Note that samples are in rows and metadata in columns.

### 1.2 - STAR/feature counts raw data   
---

#### 1.2.1. Loading the data

We copy the file with STAR/featurecounts readcounts in our personal folder from the alldata folder with the `file.copy()` function.

In [ ]:
## Code cell 19 ##

file.copy(from = paste0(gohome, "alldata/Results/featurecounts/11samples_paired-unstranded.counts"),
          to = datafolder)

We read the tabulated table and we store the data in a dataframe named `counts`.   
In addition to specifying the separator and the columns names, we also indicate that lines beginning with "#" are comments and should be ignored.

In [ ]:
## Code cell 20 ##

countdata <- read.table(paste0(datafolder, "11samples_paired-unstranded.counts"),
                     sep = "\t",
                     header = TRUE,
                     comment="#")

In [ ]:
## Code cell 21 ##

class(countdata)

In [ ]:
## Code cell 22 ##

str(countdata)

In [ ]:
## Code cell 23 ##

head(countdata)

Note that each row corresponds to a gene, with the first 6 columns providing Ensembl/Gencode gene annotations. The next columns correspond to our samples.

---
#### 1.2.2 - Counts data formatting


- **removing useless columns**

The `countdata`dataframe contains some columns which are irrelevant for our current analysis and will be removed.
These include the chromosome numbers, start and end positions, strand and gene length (columns 2 to 6). We don't need this information to perform the exploratory and differential expression analysis, hence we drop these with the following code. We however keep the geneID in the first column *(better than as row names)*. Thus, if we want to get only the columns of data counts without the geneID, we will have to use either `countdata[,-1]` or `countdata[,2:12])`.

<div class="alert alert-block alert-warning"> <b> Warning: </b><br>
<b>R is extremely memory-consuming</b>. It is not recommanded to keep redundant R objects in a session. While formating the <code>countdata</code> object in this section, and filtering it in the next one, we will replace it by itself after each step. Thus, be careful to run the cells in the right order. If you have a doubt at some point, go back to cell 13 to start again from the initial dataframe</div>

In [ ]:
## Code cell 24 ##

countdata <- countdata[, c(1,7:17)]
str(countdata)
head(countdata)

In [ ]:
## Code cell 25 ##

dim(countdata)

- **reformatting Sample names**

Below are the sample names in the counts dataframe:

In [ ]:
## Code cell 26 ##

names(countdata)

The names of the columns are not very easy to read, and most of all, they don't correspond to the sample names present in the first column of the metadata. Therefore, we cannot have any correspondance between the two information. So we modify the names of the columns in `counts`, in order to remove the first part in front of the sample number, and then the last part after the `_`. 

In [ ]:
## Code cell 27 ##

# We remove the common prefix in all columns except the first one
colnames(countdata)[-1] <- gsub("X.shared.projects.2413_rnaseq_cea.alldata.Results.featurecounts.",
                         "",
                         colnames(countdata)[-1])

# We remove the common suffix in all columns except the first one
colnames(countdata)[-1] <- gsub("_Aligned.sortedByNames.bam",
                         "",
                         colnames(countdata)[-1])

We check the new names:

In [ ]:
## Code cell 28 ##

names(countdata)

To ensure the metadata `samples` and `countdata` dataframes have the same samples names *(tested with `==`)*, we run the code below.   
This will return TRUE if the names in both files are the same. In all there are 11 samples in both files.

In [ ]:
## Code cell 29 ##

table(colnames(countdata)[-1] == samples$SampleID)

- **rename SampleID with SampleName**:

SRR names are not very informative. We replace them by the corresponding sample name.

In [ ]:
## Code cell 30 ##

colnames(countdata)[-1] <- samples$SampleName
str(countdata)
head(countdata)

### 1.3 - Salmon pseudo-mapping data   
---

In [ ]:
## Code cell 31 ##

cat("Start time is :", date(), "\n")
cat("Gencode reference : ", annot_version, "\n")  
cat("Salmon directory is ", salmonfolder, "\n")

#### 1.3.1. Getting the salmon files

We directly do it for the 11 samples after copying them to our own directory *(be aware that here we erase our previous folders)*. You can skip this cell in your own study if all your salmon results are already in your personnal directory.

In [ ]:
## Code cell 32 ##

R.utils::copyDirectory(from = paste0(gohome, "alldata/Results/salmon/"),
                       to = salmonfolder,
                       recursive = TRUE)

We now retrieve IDs of our samples: 

In [ ]:
## Code cell 33 ##

sample_names <- list.files("./Results/salmon/")
sample_names

We then list the file paths to the corresponding `quant.sf` files containing the Salmon pseudomapping counts per transcripts for each sample.

In [ ]:
## Code cell 34 ##

sample_files <- paste0("./Results/salmon/",sample_names, "/quant.sf")
sample_files

 => We check the number of samples we are dealing with:

In [ ]:
## Code cell 34 ##

cat("\nNumber of samples : ", length(sample_names), "\n")

--
#### 1.3.2. Create a table of **raw read counts** *(NumReads)* per transcript for all samples

We now start with the first sample and read the `quant.sf` file with its first column *(name of the transcript)* and its last (=5th) column *(NumReads)* containg the raw counts per transcript:

In [ ]:
## Code cell 36 ##

cat("\nFirst sample :", sample_files[1], "\n")

In [ ]:
## Code cell 37 ##

numreads <- read.table(sample_files[1], sep = "\t", header= T)
numreads <- numreads [,c(1,5)]
head(numreads)
dim(numreads)

We keep going with a loop to retrieve in the same dataframe the other samples and we rename the columns: the first one as `transcript_id` and the `NumReads` columns with the sample name.

In [ ]:
## Code cell 38 ##

for (i in 2:length(sample_files)) { 
 mytab <- read.table(sample_files[i], sep = "\t", header= T)
 mytab  <- mytab  [,c(1,5)]  
 names(mytab)[2] <- sample_names[i]
 numreads <- numreads %>% left_join(mytab, by='Name')
}
names(numreads) <- c("transcript_id", sample_names)	
head(numreads)
rm(mytab)

For further analyses, it is even better to rename the samples currently idntified by their SRR number with the corresponding sample name after checking the correct correspondance of IDs. 

In [ ]:
## Code cell 39 ##

table(names(numreads)[-1] == samples$SampleID)
names(numreads)[-1]  <- samples$SampleName
head(numreads)

--
#### 1.3.3. Create a table of **transcripts per millions** *(TPM)* for all samples

Similarly, we generate a dataframe with TPM counts for all samples. These TPMs were already calculated by salmon and are present in the forth column of the `quant.sf` files.

In [ ]:
## Code cell 40 ##

tpm <- read.table(sample_files[1], sep = "\t", header= T)
tpm <- tpm [,c(1,4)]
#head(tpm)

for (i in 2:length(sample_files)) { 
 mytab <- read.table(sample_files[i], sep = "\t", header= T)
 mytab  <- mytab  [,c(1,4)]  
 names(mytab)[2] <- sample_names[i]
 tpm <- tpm %>% left_join(mytab, by='Name')
}
names(tpm) <- c("transcript_id", sample_names)	
cat("Table dimensions : ", dim(tpm), "\n")
head(tpm)
rm(mytab)

For further analyses, it is even better to rename the samples currently idntified by their SRR number with the corresponding sample name after checking the correct correspondance of IDs. 

In [ ]:
## Code cell 41 ##

table(names(tpm)[-1] == samples$SampleID)
names(tpm)[-1]  <- samples$SampleName
head(tpm)

--
#### 1.3.4. Aggregate counts at the **gene level** for all samples

- **Upload the transcript reference:**

We need to get the transcript IDs and the corresponding gene IDs.

We read rge transcript fast gz file and select only the rows staring with an `>` to get the transcript IDs.

In [ ]:
## Code cell 42 ##

transcripts_info <- data.table::fread(paste0(reffolder, "salmon/gencode.vM35.transcripts.fa.gz"), header = FALSE) %>% filter(str_detect(V1, "^>"))
str(transcripts_info)

As you can see, the first and only column contains several information separated by a `|` so we separate this column in 8.

In [ ]:
## Code cell 43 ##

transcripts_info <- transcripts_info %>% separate(V1,
                                                  into = c("transcript_id", "gene_id", "havana_gene_id", "havana_tr_id", "ext_tr_name", "gene_name", "length", "biotype"),
                                                  remove = TRUE,
                                                 sep = "\\|",
                                                 extra ="drop")
str(transcripts_info)

Finally, we need to further format this dataframe by removing the `>` at the beginning of the ENSG transcript ID in the first column.

In [ ]:
## Code cell 44 ##

transcripts_info <- transcripts_info %>% mutate(transcript_id = str_replace(transcript_id, ">", ""))
head(transcripts_info)
tail(transcripts_info)

As a final check, we verify all the transcript ids persent in numreads (and tpm) are well present in this transcript annotation file.

In [ ]:
## Code cell 45 ##

cat("\nCheck missing transcripts...\n")
cat(" this experiment - the reference :")
cat(length(setdiff(numreads$transcript_id, transcripts_info$transcript_id)), "\n")
# [1] 0
cat(" the reference - this experiment :")
cat(length(setdiff(transcripts_info$transcript_id, numreads$transcript_id)), "\n")
# [1] 847


we are now ready to perform the aggregation by genes, since the transcript annotation file contains both the transcripts and the gene ids.

##### **a-gene aggregation on numreads**

In [ ]:
## Code cell 46 ##

genecounts <- numreads  %>% left_join(transcripts_info %>% select(transcript_id, gene_id),
                                      by='transcript_id') %>% 
  select(transcript_id, gene_id, everything()) %>% select(-1) %>% 
  group_by(gene_id) %>% summarize_all(sum, na.rm = TRUE) 

#genecounts <- genecounts %>% mutate(gene_id = str_replace(gene_id, "\\.\\d{1,2}$", "")) #in case you want to remove the .XX in the gene_id

cat("Table dimensions:", dim(genecounts), "\n")

head(genecounts)

We add gene names/symbol for further analyses

In [ ]:
## Code cell 47 ##

genecounts_symbols <- genecounts %>% left_join(transcripts_info %>%
                                   mutate(gene_id = str_remove(gene_id, ".\\d{1,2}$")) %>%  
 select(gene_id, gene_name), by = 'gene_id') %>% select(-gene_id) %>% 
        select(gene_name, everything()) %>% distinct()

cat("Table dimensions:", dim(genecounts_symbols), "\n")

head(genecounts_symbols)

##### **b-gene aggregation on TPMs**

We do not recommand to keep working with gene levels on TPMs, but for a quick and dirty evaluation of gene levels, we might want them.

In [ ]:
## Code cell 48 ##

gtpm <- tpm  %>% left_join(transcripts_info %>% select(transcript_id, gene_id), by='transcript_id') %>% 
	select(transcript_id, gene_id, everything()) %>% select(-1) %>% 
	group_by(gene_id) %>% summarize_all(sum, na.rm = TRUE) 

#gtpm <- gtpm %>% mutate(gene_id = str_replace(gene_id, "\\.\\d{1,2}$", "")) #in cas eyou want to remove the .XX in the gene_id

cat("Table dimensions:", dim(gtpm), "\n")

head(gtpm)

and we add gene names :

In [ ]:
## Code cell 49 ##

gtpm_symbols <- gtpm %>% left_join(transcripts_info %>%
                                   mutate(gene_id = str_remove(gene_id, ".\\d{1,2}$")) %>%  
 select(gene_id, gene_name), by = 'gene_id') %>% select(-gene_id) %>% 
        select(gene_name, everything()) %>% distinct()

cat("Table dimensions:", dim(gtpm_symbols), "\n")

head(gtpm_symbols)

In [ ]:
## Code cell 50 ##

cat("End time is :", date(), "\n")

---

## 2 - Filtering out non informative genes

---

For many analysis methods, it is advisable to filter out as many genes as possible before the analysis to decrease the impact of multiple testing correction on false discovery rates.   

This is normally done by filtering out **genes with low numbers of reads** and thus likely to be uninformative from a biological point of view.  
(With DESeq2, the tool that we will use later for differential analysis, this filtering is not necessary beforehand, as DESeq2 applies independent filtering during the analysis.)   
In addition, filtering out genes that are very lowly expressed does reduce the size of the dataset, meaning that less memory is required and processing steps are carried out faster.

<div class="alert alert-block alert-info"> <b> What is a lowly expressed gene ? </b><br>
A general rule is to keep genes with a read count across all samples equal or above to number of samples.</div>

For some biological questions, we may want to **keep only coding genes with or without long-non-coding RNAs**, thus also getting rid of small RNAs, microRNAs, pseudogenes, tRNAs, Ig genes, TCR genes, ..

=> Here, we decide to keep all gene categories but to filter on the level of expression. We thus keep genes where the total number of reads across all 11 samples is greater than 10.

### 2.1. On STAR/feature counts data

- For each gene, we compute the total count of read in all samples, and compare to our threshold. We count the number of genes passing the test.

In [ ]:
## Code cell 51 ##

# keeping outcome in a vector of 'logical' values (ie TRUE or FALSE, or NA)
keep <- rowSums(countdata[-1]) > 10

# summary of test outcome: number of genes in each class:
table(keep, useNA="always") 

This means that 32754 genes have a total count across samples below 10. We keep the other 24432 genes.  
For information, when we performed the same analysis in 2023 with the M32 annotation, we eliminated 34083 genes with a total count across samples below 10 and we kept 22927 genes

- We extract those genes and replace `counts` with this subset of genes.

In [ ]:
## Code cell 52 ##


# subset genes where test was TRUE
countdata <- countdata[keep,]
rm(keep)

# check dimension of new count matrix
dim(countdata)

# 24432 12

<div class="alert alert-block alert-warning"> <b> Warning: </b><br>
If your dataframe after cell 29 does not have 24432 rows and 12 columns, you probably run a command twice. In that case, rerun all cells from cell 17.</div>

### 2.2. On Salmon data

We apply the same strategy as for STAR/featuredounts data at the gene level.

In [ ]:
## Code cell 53 ##

# keeping outcome in a vector of 'logical' values (ie TRUE or FALSE, or NA)
keep <- rowSums(genecounts[-1]) > 10

# summary of test outcome: number of genes in each class:
table(keep, useNA="always") 

In [ ]:
## Code cell 54 ##


# subset genes where test was TRUE
genecounts <- genecounts[keep,]
rm(keep)

# check dimension of new count matrix
dim(genecounts)

# 21427 12


<div class="alert alert-block alert-warning"> <b> Warning: </b><br>
    We see the number of genes kept is slightly smaller than with STAR/featurecounts. </div>

Let's see whether they are all part of STAR/featureCounts analysis:

In [ ]:
## Code cell 55 ##

cat("----number of common genes:\n")

shared_star_salmon <- intersect(genecounts$gene_id, countdata$Geneid)
length(shared_star_salmon)

cat("----number of genes only in STAR:\n")
length(setdiff( countdata$Geneid, genecounts$gene_id))

cat("----number of genes only in Salmon:\n")
length(setdiff(genecounts$gene_id, countdata$Geneid))

---
## 3 - Quality assessment
---

Before moving on to the actual differential expression analysis, it is important to assess the quality of our data.   
Quality control is an important step in any data analysis. Since our purpose is to identify differentially expressed genes, we may consider excluding a sample if it is an obvious outlier that could reflect an issue in its upstream processing or preparation or if it is inconsistent with metadata (eg. Y chromosome genes expressed in females).

In the ensuing steps, we will explore our data using boxplots, PCA, density plots and heatmaps.


### 3.1 - Initial visualization of the count distributions
---

Differential expression calculations with DESeq2 uses raw read counts as input, but for visualization purposes we use transformed counts.

Why not raw counts? Two issues:

    - Raw counts range is very large: some highly expressed genes can have hundreds of thousands of reads
    - Variance increases with mean gene expression, and this has impact on assessing the relationships.
    
Let's display the range of expression of our raw read counts:

- for **STAR/featurecounts**

In [ ]:
## Code cell 56 ##

summary(countdata[,-1])

- for **Salmon aggregated gene counts**:

In [ ]:
## Code cell 57 ##

summary(genecounts[,-1])

We also can see that few outlier genes affect distribution visualization:

- for STAR/featurecounts:

In [ ]:
## Code cell 58 ##

opar <- par(no.readonly=TRUE) # l'argument no.readonly=TRUE permet de supprimer l'affichage d'éventuels warnings 
par(mar = c(8,2,2,2)) # to increase margin at the bottom to display full sample names
boxplot(countdata[,-1], main = 'STAR/featurecounts raw readcounts distribution across samples', xaxt = "n") # no display of xlabels
text(1:11, y = par("usr")[3] - 0.45,
     labels = names(countdata)[-1],
     srt = 30, adj = 1, xpd = NA, cex= 0.9)# add xlabels with 30 degres srotation
suppressWarnings(par(opar))


- for Salmon aggregated gene counts:

In [ ]:
## Code cell 59 ##

opar <- par(no.readonly=TRUE) # l'argument no.readonly=TRUE permet de supprimer l'affichage d'éventuels warnings 
par(mar = c(8,2,2,2)) # to increase margin at the bottom to display full sample names
boxplot(genecounts[,-1], main = 'Salmon gene raw readcounts distribution across samples', xaxt = "n") # no display of xlabels
text(1:11, y = par("usr")[3] - 0.45,
     labels = names(countdata)[-1],
     srt = 30, adj = 1, xpd = NA, cex= 0.9)# add xlabels with 30 degres srotation
suppressWarnings(par(opar))


We can see that the variance increases with the average level of expression by plotting standard deviation vs mean expression :

In [ ]:
## Code cell 60 ##

opar <- par(no.readonly=TRUE)
par(mfrow = c(1,2))
# Raw counts mean expression vs Standard Deviation (SD) with STAR/featurecounts
plot(rowMeans(countdata[,-1]), apply(countdata[,2:12], 1, sd, na.rm = TRUE), 
     main = 'STAR/feature counts Raw read counts: sd vs mean', 
     xlim = c(0,100000), # to zoom
     ylim = c(0,50000), # to zoom
     xlab = "Mean of raw counts per gene",
     ylab = "SD of raw counts per gene"
    )
# Raw counts mean expression vs Standard Deviation (SD) with Salmon
plot(rowMeans(genecounts[,-1]), apply(genecounts[,2:12], 1, sd, na.rm = TRUE), 
     main = 'Salmon raw read counts: sd vs mean', 
     xlim = c(0,100000), # to zoom
     ylim = c(0,50000), # to zoom
     xlab = "Mean of raw counts per gene",
     ylab = "SD of raw counts per gene"
    )
par(opar) 

This problem is called ***heteroskedasticy*** : variances and means are not independent.

### 3.2 -  Data transformation and visualization
---

In exploratory data analysis, we are going to look at the distance between samples. It is thus crucial to make the data homoscedatic.

To avoid the problems posed by raw counts, they can be transformed. **Several transformation methods exist to limit the dependence of variance on mean gene expression**, among which :

    - Simple log2 transformation
    - VST : variance stabilizing transformation
    - rlog : regularized log transformation

For the moment, we are going to use only the first one to perform this first exploratory analysis on non-normalised data.   
The two other types of transformation are available in the DESeq2 package, that we will use in the next jupyter notebook.  
 

#### **-> log2 transformation**

This is one of the most used transformation in transcriptomics, as it helps to normalize the data (with the meaning of changing the data distribution to one closer to the normal distribution), and it enables a better visualization of low counts.   
    
Because some genes are not expressed (detected) in some samples, their counts are 0. As log2(0) equals -Inf, we add an ***offset***, usually of 1, to every count value to create **pseudocounts** (nothing to do with Salmon pseudomapping). The lowest value then is 1, or 0 on the log2 scale (log2(1) = 0).

- for STAR/featurecounts:

In [ ]:
## Code cell 61 ##

summary(log2(countdata[,2]+1)) # summary for first sample column 2

In [ ]:
## Code cell 62 ##

summary(log2(countdata[,2:12]+1)) # summary for each sample

- for Salmon

In [ ]:
## Code cell 63 ##

summary(log2(genecounts[,2:12]+1)) # summary for each sample

We will check the distribution of read pseudocounts using a boxplot and add some colour to see if there is any difference between sample groups.

- for STAR/featurecounts

In [ ]:
## Code cell 64 ##

# make a colour vector
conditionColor <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# Check distributions of samples using boxplots
opar <- par(no.readonly=TRUE) # l'argument no.readonly=TRUE permet de supprimer l'affichage d'éventuels warnings 
par(mar = c(6,2,2,2))
boxplot(log2(countdata[,2:12]+1),
        xlab = "", xaxt = "n",
        ylab = "Log2(Counts)",
        las = 2,
        col = conditionColor,
        main = "Log2( STAR/featurecounts Counts) distribution across samples")
text(1:11, y = par("usr")[3] - 0.45,
     labels = names(countdata)[-1],
     srt = 30, adj = 1, xpd = NA, cex= 0.9)


# Let's add a blue horizontal line that corresponds to the median
abline(h = median.default(as.matrix(log2(countdata[,2:12]+1))), col="blue")

suppressWarnings(par(opar))

- for Salmon :

In [ ]:
## Code cell 65 ##

# Check distributions of samples using boxplots
opar <- par(no.readonly=TRUE) # l'argument no.readonly=TRUE permet de supprimer l'affichage d'éventuels warnings 
par(mar = c(6,2,2,2))
boxplot(log2(genecounts[,2:12]+1),
        xlab = "", xaxt = "n",
        ylab = "Log2(Counts)",
        las = 2,
        col = conditionColor,
        main = "Log2(STAR Counts) distribution across samples")
text(1:11, y = par("usr")[3] - 0.45,
     labels = names(genecounts)[-1],
     srt = 30, adj = 1, xpd = NA, cex= 0.9)

# Let's add a blue horizontal line that corresponds to the median
abline(h = median.default(as.matrix(log2(genecounts[,2:12]+1))), col="blue")

suppressWarnings(par(opar))

From the boxplot, we see that overall the density distributions of raw log-counts are not identical but still not very different from one sample to another.   
If a sample is really far above or below the blue horizontal line (overall median) we may need to investigate that sample further.

In [ ]:
## Code cell 66 ##

# Log2 counts standard deviation (sd) vs mean expression

opar <- par(no.readonly=TRUE)
par(mfrow = c(1,2))
plot(rowMeans(log2(countdata[,2:12]+1)),
              matrixStats::rowSds(as.matrix(log2(countdata[,2:12]+1), na.rm = TRUE)), 
     main = 'Log2 STAR/Featurecounts Counts: sd vs mean',
     xlab = "Mean of log2(raw counts) per gene",
     ylab = "SD of log2(raw counts) per gene"
    
    )
plot(rowMeans(log2(genecounts[,2:12]+1)),
              matrixStats::rowSds(as.matrix(log2(genecounts[,2:12]+1), na.rm = TRUE)), 
     main = 'Log2 STAR/Featurecounts Counts: sd vs mean',
     xlab = "Mean of log2(raw counts) per gene",
     ylab = "SD of log2(raw counts) per gene"
    
    )
par(opar) 

In contrast to raw counts, with log2 transformed counts, lowly expressed genes (mean expression around 5 or below) show higher variation than highly expressed genes (mean expression above 10-12).

### 3.3 - Principal Component Analysis (PCA)
---

A principal component analysis (PCA) is an unsupervised method used to explore the data variance structure **by reducing its dimensions to a few principal components (PC) that explain the greatest variation in the data**.  PCA is an example of an unsupervised analysis, where we don’t specify the grouping of the samples *(see lecture 12)*.  
If the experiment is well controlled and has worked properly, we should find that replicate samples cluster closely, whilst the greatest sources of variation in the data should be between treatments/sample groups.   
It is also an incredibly useful tool for checking for outliers and batch effects.

We cannot run the PCA directly on raw non-normalized data. Data must have a common scale. To circumvent this, we thus run here the PCA on the log2 transformation data. Another method could have been to center (to substract the mean) and reduce (to divide by the variance). 

#### 3.3.1 with STAR/faturecounts data:

- transposition of data : 

In a PCA where we want to visualize the grouping of samples, the variables are genes, and thus should be in columns. So in the code below, the `t()` function is used to transpose the dataframe: the result is a new dataframe with samples as rows and genes as columns.


In [ ]:
## Code cell 67 ##

tlogcounts_star <- t(log2(countdata[,2:12]+1))
dim(tlogcounts_star)

- plot PCA:

To plot the PCA results, we will use here the `autoplot` function from the `ggfortify` package (Tang, Horikoshi, and Li 2016). `ggfortify` is built on top of `ggplot2` and is able to recognise common statistical objects such as PCA results or linear models and to automatically generate a summary plot of the results in an appropriate manner.

In [ ]:
## Code cell 68 ##

# run PCA
PCAdata <- prcomp(tlogcounts_star)

# plot PCA
autoplot(PCAdata)


Without the names of the samples, or colours indicating their group, we cannot see if they cluster correctly.    

- add labels to PCA plot:

So we add colours and labels to the PCA plot.   
The package `ggrepel` allows us to add text to the plot, but ensures that points that are close together don’t have overlapping labels  (they repel each other).

In [ ]:
## Code cell 69 ##

autoplot(PCAdata,
         data = samples, 
         colour = "Condition", 
         shape = "Tissue",
         size = 6) +
    geom_text_repel(aes(x = PC1, y = PC2, label = SampleName),
                        box.padding = 0.8)


We can see that the first Principal Component (explaining the largest source of variation) shows in this dataset variation between samples from different conditions (the effect of interest), while the second PC (explaining the second largest source of variation) displays here sample differences due to WT vs mutant genotypes.   
It seems that there is no batch effect, but let's verify if none appears in the next PCs eigen vectors.  

In [ ]:
## Code cell 70 ##

autoplot(PCAdata,
         x = 2,    # PC2
         y = 3,    # PC3
         data = samples, 
         colour = "Condition", 
         shape = "Tissue",
         size = 6) +
    geom_text_repel(aes(x = PC2, y = PC3, label = SampleName),
                    box.padding = 0.8)


In [ ]:
## Code cell 71 ##

autoplot(PCAdata,
         x = 3,    # PC3
         y = 4,    # PC4
         data = samples, 
         colour = "Condition", 
         shape = "Tissue",
         size = 6) +
    geom_text_repel(aes(x = PC3, y = PC4, label = SampleName),
                    box.padding = 0.8)

rm(PCAdata) # we remove the PCA dtaa from the session for memory reasons

#### 3.3.2 with Salmon gene levels data:

quickly...

In [ ]:
## Code cell 72 ##

tlogcounts_salmon <- t(log2(genecounts[,2:12]+1))

PCAdata <- prcomp(tlogcounts_star)

autoplot(PCAdata,
         data = samples, 
         colour = "Condition", 
         shape = "Tissue",
         size = 6) +
    geom_text_repel(aes(x = PC1, y = PC2, label = SampleName),
                        box.padding = 0.8)


On the basis of these last two plots, we cannot see a clear effect of a given factor to the variation depicted in PC3 and PC4, except maybe the difference between mice.

*Note: We will perform a more detailed PCA after data normalisation in notebook Pipe_10.*

### 3.4 - Hierarchical clustering
---

This  representation can also be used to cluster the samples based on dissimilarity indexes *(see lecture 13, here we will use Ward distance)*. More information can be found with `?hclust` (or in the Contextual Help panel on the right, that can be opened via the Help menu).

- with STAR/featurecounts:

In [ ]:
## Code cell 73 ##

clusters <- hclust(dist(as.matrix(tlogcounts_star)), method ="ward.D")
plot(clusters, labels = samples$SampleName)

rm(clusters, tlogcounts_star)    

In [ ]:
## Code cell 74 ##

clusters <- hclust(dist(as.matrix(tlogcounts_salmon)), method ="ward.D")
plot(clusters, labels = samples$SampleName)

rm(clusters, tlogcounts_salmon)    

We can see that our samples are grouped correctly, and that replicates from the same mouse cluster together, as expected.

### 3.5 - Density plot
---

A Density Plot can be used to visualize the distribution of data over a continuous interval. In RNA-seq analysis, this could be used to detect the presence or absence of batch effects in the data. Batch effects may be introduced through different experimental platforms, laboratory conditions, different sources of samples, different technicians, etc, and may introduce spurious variability which is not due to the condition under study (cancerous state of B and pro-B cells).   
This [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4636836/) comprehensively discusses batch effects and how they can be corrected.

- for STAR/featurecounts:

In [ ]:
## Code cell 75 ##

affy::plotDensity(log2(countdata[,2:12]+1),
                  xlab("Density"),ylab("log2(Counts)"),
                  col = 1:11)
legend(x = 14, y = 0.13,legend = names(log2(countdata[,2:12]+1)),
       col = 1:11, lty = 1:11, bty = "n")

- for Salmon:

In [ ]:
## Code cell 76 ##

affy::plotDensity(log2(genecounts[,2:12]+1),
                  xlab("Density"),ylab("log2(Counts)"),
                  col = 1:11)
legend(x = 14, y = 0.13,legend = names(log2(genecounts[,2:12]+1)),
       col = 1:11, lty = 1:11, bty = "n")

The density plots overlap for all samples, none stand out as having a different profile.   
This confirms that no batch effect is visible in our data that would need to be corrected before going on with normalisation.   

To see examples of batch effects, you can have a look [there](https://evayiwenwang.github.io/Managing_batch_effects/detect.html). 

### 3.6 - Correlation between STAR/featurecounts and Salmon counts for common genes:

In [ ]:
## Code cell 77 ##

temp1 <- pivot_longer(subset(countdata, Geneid %in% shared_star_salmon),cols=2:12, names_to = "sample", values_to = "counts")
temp2 <- pivot_longer(subset(genecounts, gene_id %in% shared_star_salmon),cols=2:12, names_to = "sample", values_to = "counts")
temp1 <- temp1[order(temp1$Geneid),]
temp2 <- temp2[order(temp2$gene_id),]
cor.test(temp1$counts, temp2$counts)
plot(log2(temp1$counts+1), log2(temp2$counts+1))

rm(temp1, temp2)

<div class="alert alert-block alert-danger"><b>Info:</b> Fot the following notebooks, we will work only on STAR data but you can easily adapt the future notebooks using the salmon count data.</div>

## 4 - Saving our results

We can save all the R objects created in this session in a single R object.   
This will help us to reload our dataframes without having to run the same commands.   

In [ ]:
## Code cell 78 ##  

ls()
save(countdata, genecounts, genecounts_symbols, gtpm, gtpm_symbols, samples, conditionColor, file = paste0(pca1folder,"RawCounts_Samples.RData"))

---
___

Now we go on with the normalisation of read counts and differential expression analysis using `DESeq2`.  
  
**=> Step 9: DESeq2 Normalisation and Differential Expression analysis** 

The jupyter notebook used for the next session will be *Pipe_09-R-DESeq2-normalisation-DE.ipynb*    
Let's retrieve it in our directory, in order to have a private copy to work on:   



**Save executed notebook**

To end the session, save your executed notebook in your `run_notebooks` folder. **Adjust the name with yours** and reformat as code cell to run it.

In [ ]:
## Code cell 80 ##   

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste0(myfolder,"/run_notebooks"), recursive = TRUE)

runfolder <- paste0(myfolder,"/run_notebooks")
       
# file.copy(paste0(myfolder, "/Pipe_08-R_counts-exploratory-analysis-I.ipynb"), runfolder)
file.copy(paste0(myfolder, "/Pipe_08-R_counts-exploratory-analysis-I.ipynb"), paste0(runfolder, "/Pipe_08-R_counts-exploratory-analysis-I-run.ipynb"))

---

<div class="alert alert-block alert-success"><b>Success:</b> Well done! You now know how to perform a exploratory analysis of RNAseq expression data in R.<br>
Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to the server! 
</div>

## Useful commands
<div class="alert alert-block alert-info"> 
    
- <kbd>CTRL</kbd>+<kbd>S</kbd> : save notebook<br>    
- <kbd>CTRL</kbd>+<kbd>ENTER</kbd> : Run Cell<br>  
- <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> : Run Cell and Select Next<br>   
- <kbd>ALT</kbd>+<kbd>ENTER</kbd> : Run Cell and Insert Below<br>   
- <kbd>ESC</kbd>+<kbd>y</kbd> : Change to *Code* Cell Type<br>  
- <kbd>ESC</kbd>+<kbd>m</kbd> : Change to *Markdown* Cell Type<br> 
- <kbd>ESC</kbd>+<kbd>r</kbd> : Change to *Raw* Cell Type<br>    
- <kbd>ESC</kbd>+<kbd>a</kbd> : Create Cell Above<br> 
- <kbd>ESC</kbd>+<kbd>b</kbd> : Create Cell Below<br> 

<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>   
</em>    
 
</div>

Claire Vandiedonck - 2021-2023   
Sandrine Caburet - 05/2023   
MAJ : 23/06/2024 by @CVandiedonck